In [ ]:
#instalamos los archivos correspondientes.
!apt-get install -y ffmpeg
!pip install pydub

#ahora llamamos a las librerias que vamos a ocupar
# Importamos las librerías
import librosa
import librosa.display
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import gc
from google.colab import drive
from pydub import AudioSegment
from pydub.utils import mediainfo
import subprocess
import cv2


# Función principal para generar ambos DataFrames
def procesar_espectrogramas(ruta_carpeta_IA, ruta_carpeta_reales, ruta_salida_IA, ruta_salida_reales):
    df_IA = obtener_data_audio(ruta_carpeta_IA, ruta_salida_IA, etiqueta=0)
    df_reales = obtener_data_audio(ruta_carpeta_reales, ruta_salida_reales, etiqueta=1)
    return df_IA, df_reales

# Función para convertir un archivo FLAC a WAV
def convertir_flac_a_wav(archivo_flac):
    if not archivo_flac.endswith('.flac'):
        return None
    archivo_wav = archivo_flac.replace('.flac', '.wav')
    try:
        subprocess.run(['ffmpeg', '-y', '-i', archivo_flac, archivo_wav], check=True)
        return archivo_wav
    except subprocess.CalledProcessError:
        return None

# Función para convertir un archivo de audio en espectrograma y guardarlo como imagen
def convertir_y_guardar_espectrograma(archivo_audio, nombre_salida, carpeta_espectro):
    ruta_imagen = os.path.join(carpeta_espectro, nombre_salida)
    if os.path.exists(ruta_imagen):  # Verificar si la imagen ya existe
        return ruta_imagen
    try:
      y, sr = librosa.load(archivo_audio, sr=None, duration=60)

    except Exception as e:
        # Explicitly specify format and subtype when loading with soundfile
        try:
            y, sr = sf.read(archivo_audio, dtype='float32')
        except sf.LibsndfileError as e:
            print(f"Error loading {archivo_audio}: {e}")
            return None # Skip this file if it can't be loaded



    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=64)
    S_db = librosa.power_to_db(S, ref=np.max)

    plt.figure(figsize=(5, 2))
    librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='mel')
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(ruta_imagen, format='png', bbox_inches='tight', pad_inches=0)
    plt.close()
    gc.collect()

    return ruta_imagen

# Función para procesar audios y generar un DataFrame
def obtener_data_audio(ruta_carpeta, ruta_salida, etiqueta):
    nombres_archivos = []
    duraciones = []
    frecuencias_muestreo = []
    rutas_imagenes_espectrograma = []
    etiquetas = []

    if not os.path.exists(ruta_salida):
        os.makedirs(ruta_salida)

    archivos_audio = [f for f in os.listdir(ruta_carpeta) if f.endswith(('.mp3', '.wav', '.flac'))]

    for archivo in archivos_audio:
        ruta_archivo = os.path.join(ruta_carpeta, archivo)
        if archivo.endswith('.flac'):
            ruta_archivo = convertir_flac_a_wav(ruta_archivo)
            if not ruta_archivo:
                continue

        y, sr = librosa.load(ruta_archivo, sr=None, duration=60)
        nombres_archivos.append(archivo)
        duraciones.append(librosa.get_duration(y=y, sr=sr))
        frecuencias_muestreo.append(sr)

        nombre_salida_espectrograma = archivo.replace('.mp3', '.png').replace('.wav', '.png').replace('.flac', '.png')
        ruta_imagen = convertir_y_guardar_espectrograma(ruta_archivo, nombre_salida_espectrograma, ruta_salida)
        rutas_imagenes_espectrograma.append(ruta_imagen)
        etiquetas.append(etiqueta)

    return pd.DataFrame({
        'Nombre del archivo': nombres_archivos,
        'Duración (segundos)': duraciones,
        'Frecuencia de muestreo': frecuencias_muestreo,
        'Ruta espectrograma': rutas_imagenes_espectrograma,
        'Etiqueta': etiquetas
    })


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Definir rutas necesarias
ruta_carpeta_IA = '/content/drive/MyDrive/Proyecto SIC/Voces IA'
ruta_carpeta_reales = '/content/drive/MyDrive/Proyecto SIC/Voces reales'
carpeta_salida_espectrogramas_IA = '/content/drive/MyDrive/Proyecto SIC/espectrogramas_IA/'
carpeta_salida_espectrogramas_reales = '/content/drive/MyDrive/Proyecto SIC/espectrogramas_reales/'


Mounted at /content/drive


In [ ]:

# Generar los DataFrames
df_IA, df_reales = procesar_espectrogramas(ruta_carpeta_IA, ruta_carpeta_reales, carpeta_salida_espectrogramas_IA, carpeta_salida_espectrogramas_reales)

# Guardar los DataFrames en archivos CSV
df_IA.to_csv('/content/drive/MyDrive/Proyecto SIC/datos_espectrogramas_IA.csv', index=False)
df_reales.to_csv('/content/drive/MyDrive/Proyecto SIC/datos_espectrogramas_reales.csv', index=False)

In [ ]:
#vamos a verificar que todos los archivos solo tengan .wav y no tengan mas puntos en caso de tener mas puntos se cambian a _
def reemplazar_puntos_en_nombres(carpeta):
    """
    Reemplaza los puntos en los nombres de los archivos por guiones bajos,
    excepto el punto antes de la extensión '.wav'.

    Args:
        carpeta (str): Ruta de la carpeta donde se procesarán los archivos.
    """
    for raiz, _, archivos in os.walk(carpeta):
        for archivo in archivos:
            # Verificar si el archivo tiene la extensión .wav
            if archivo.endswith('.wav'):
                # Separar nombre y extensión
                nombre, extension = os.path.splitext(archivo)

                # Reemplazar puntos en el nombre, dejando intacta la extensión
                nuevo_nombre = nombre.replace('.', '_') + extension

                # Construir rutas completas
                ruta_actual = os.path.join(raiz, archivo)
                nueva_ruta = os.path.join(raiz, nuevo_nombre)

                # Renombrar archivo
                os.rename(ruta_actual, nueva_ruta)
                print(f"Renombrado: {ruta_actual} -> {nueva_ruta}")

# Llamar a la función para cada carpeta
reemplazar_puntos_en_nombres(ruta_carpeta_IA)
reemplazar_puntos_en_nombres(ruta_carpeta_reales)

Renombrado: /content/drive/MyDrive/Proyecto SIC/Voces IA/Ibai's Voice(2).wav -> /content/drive/MyDrive/Proyecto SIC/Voces IA/Ibai's Voice(2).wav
Renombrado: /content/drive/MyDrive/Proyecto SIC/Voces IA/Ibai's Voice(3).wav -> /content/drive/MyDrive/Proyecto SIC/Voces IA/Ibai's Voice(3).wav
Renombrado: /content/drive/MyDrive/Proyecto SIC/Voces IA/Ibai's Voice(4).wav -> /content/drive/MyDrive/Proyecto SIC/Voces IA/Ibai's Voice(4).wav
Renombrado: /content/drive/MyDrive/Proyecto SIC/Voces IA/Ibai's Voice(5).wav -> /content/drive/MyDrive/Proyecto SIC/Voces IA/Ibai's Voice(5).wav
Renombrado: /content/drive/MyDrive/Proyecto SIC/Voces IA/Ibai's Voice(6).wav -> /content/drive/MyDrive/Proyecto SIC/Voces IA/Ibai's Voice(6).wav
Renombrado: /content/drive/MyDrive/Proyecto SIC/Voces IA/clip_1.wav -> /content/drive/MyDrive/Proyecto SIC/Voces IA/clip_1.wav
Renombrado: /content/drive/MyDrive/Proyecto SIC/Voces IA/clip_2.wav -> /content/drive/MyDrive/Proyecto SIC/Voces IA/clip_2.wav
Renombrado: /content/